In [2]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model


In [4]:
#live video

classifier = cv2.CascadeClassifier(f'haarcascade_frontalface_default.xml')

emotion_model = load_model('bestweight.h5')


emotion_dim = (48,48)

emotion_bw = False 

emotion_dict = {0: 'angry', 1: 'happy', 2: 'neutral', 3: 'sad', 4: 'disgust', 5: 'fear', 6: 'surprise'}

vid_frames = []
cap = cv2.VideoCapture(0)
cap_video = False

if cap_video == True:
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float
    height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('Tests/FaceDetector.mp4', fourcc,10, (int(width), int(height)))
while True: 
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1,1)
    clone = frame.copy()
    bboxes = classifier.detectMultiScale(clone)
    for i in bboxes: 
        x, y, width, height = i[0], i[1], i[2], i[3]
        x2, y2 = x+ width, y+height
        emotion_roi = clone[y:y2, x:x2]
        emotion_roi= cv2.cvtColor(emotion_roi, cv2.COLOR_BGR2GRAY)
        emotion_roi = cv2.resize(emotion_roi, emotion_dim, interpolation = cv2.INTER_CUBIC)
        emotion_roi = emotion_roi/255
        emotion_roi = emotion_roi.reshape(1, emotion_roi.shape[0], emotion_roi.shape[1], 1)
        #emotion predictions
        emotion_predict = emotion_model.predict(emotion_roi)[0]
        emotion_idx = np.argmax(emotion_predict)
        emotion_cat = emotion_dict[emotion_idx]
        emotion_conf = f'{round(np.max(emotion_predict)*100)}%'
        cv2.putText(clone, f'{emotion_cat}: {emotion_conf}', (x, y+15), cv2.FONT_HERSHEY_SIMPLEX, .5, (0,0,225), 2)
        cv2.rectangle(clone, (x,y), (x2,y2), (0,0,225), 1)
        
      
        
        
    cv2.imshow('LIVE', clone)
    vid_frames.append(clone)
    if cap_video == True:
        out.write(clone)
    if cv2.waitKey(1) & 0xFF ==ord('q'): 
        break 
cap.release()

cv2.destroyAllWindows()

if cap_video == True:
    out.release()

In [5]:
img = cv2.imread('face.jpeg',cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (48,48), interpolation = cv2.INTER_CUBIC)
img = img/255

In [7]:
img.shape
img = img.reshape(1, img.shape[0], img.shape[1], 1)

In [10]:
emotion_model = load_model('bestweight.h5')
emotion_model.predict(img)
grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

array([[0.4053291 , 0.35004976, 0.2446211 ]], dtype=float32)